In [ ]:
! pip install opencv-python\
# import necessary libraries


In [40]:
! bash ./download.sh




2019-12-15 17:53:41 URL:http://vis-www.cs.umass.edu/lfw/lfw-a.tgz [14486641/14486641] -> "lfw-a.tgz" [1]


Verifying checksums.

+ lfw-a.tgz: successfully checked



In [187]:
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml

--2019-12-15 18:26:05--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.120.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.120.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930127 (908K) [text/plain]
Saving to: ‘haarcascade_frontalface_default.xml’

haarcascade_frontal 100%[===================>] 908.33K  --.-KB/s    in 0.09s   

2019-12-15 18:26:06 (9.59 MB/s) - ‘haarcascade_frontalface_default.xml’ saved [930127/930127]



In [6]:
import pandas as pd 
import numpy
import matplotlib.pyplot as plt 
import glob
from pyspark.sql import SparkSession
# create sparksession
spark = SparkSession \
    .builder \
    .appName("Pysparkexample") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
import cv2
from PIL import Image
import os
from time import time
import numpy as np

In [7]:
def read_image(filename):
    img_BGR = cv2.imread(filename)
    img_BGR = img_BGR.astype(np.uint8)
    return img_BGR


def grayscale(img_BGR):
    img_grayscale = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2GRAY)
    return img_grayscale


def process(filename, faceCascade, scale_factor=1.1, min_Neighbors=5, min_Size=(30,30)):
    img_BGR = read_image(filename)
    img_grayscale = grayscale(img_BGR)
    # Detect faces in the image
    faces = faceCascade.detectMultiScale(img_grayscale, scaleFactor=scale_factor, minNeighbors=min_Neighbors, minSize=min_Size)
    return faces

def find_these_files(directory):
    roots = set()
    dirs = set()
    files = set()
    for(root_folder, directory_list, file_list) in os.walk(directory):
        roots.add(root_folder)
        for directory in directory_list:
            dirs.add(directory)
        for file in file_list:
            abspath = os.path.join(root_folder, file)
            files.add(abspath)
    
    df = pd.DataFrame({ "Path" : list(files)})
    return df


def test_haar_cascades(df, cascades):
    results = pd.DataFrame()
    i = 0
    j = 0
    t0 = time()

    results = []

    for cascade in cascades:
        i = i + 1
        j = 0
        print("-----------------------------------------")
        print(i, " of ", len(cascades), " in outer loop.")
        print("-----------------------------------------")
        cascade_classifier = cv2.CascadeClassifier(cascade)
        for path in df['Path']:
            path = "/" + path
            j = j + 1
            if j == 1 or j % 10 == 0:
                print("\t", j, " of ", len(df['Path']), " in inner loop")
                pass
            try:
                result = len(process(path, cascade_classifier))
                tmp = [result, cascade, path]
                results.append(tmp)
            except:
                pass
    
    results = np.array(results)
    results = pd.DataFrame(results)
    results = results.rename(columns = {0: "Faces", 
                              1: "Cascade", 
                              2: "Image"})
    print("\t", j, " of ", len(df['Path']), " in inner loop")
    return(results)
  

In [11]:
directory = "./lfw-subset/raw/"
cascade = 'haarcascade_frontalface_default.xml'


def find_faces(directory, cascade):
    paths = find_these_files(directory)
    df = pd.DataFrame(paths)
    results = test_haar_cascades(df, [cascade])
    return df

results = find_faces(directory, cascade)
results

-----------------------------------------
1  of  1  in outer loop.
-----------------------------------------
	 1  of  35  in inner loop
	 10  of  35  in inner loop
	 20  of  35  in inner loop
	 30  of  35  in inner loop
	 35  of  35  in inner loop


,Path
0,./lfw-subset/raw/Ann_Veneman/Ann_Veneman_0004.jpg
1,./lfw-subset/raw/Ann_Veneman/Ann_Veneman_0009.jpg
2,./lfw-subset/raw/Anna_Kournikova/Anna_Kourniko...
3,./lfw-subset/raw/Ann_Veneman/Ann_Veneman_0006.jpg
4,./lfw-subset/raw/Adrien_Brody/Adrien_Brody_000...
5,./lfw-subset/raw/Anna_Kournikova/Anna_Kourniko...
6,./lfw-subset/raw/Adrien_Brody/Adrien_Brody_000...
7,./lfw-subset/raw/Anna_Kournikova/Anna_Kourniko...
8,./lfw-subset/raw/Adrien_Brody/Adrien_Brody_000...
9,./lfw-subset/raw/Ann_Veneman/Ann_Veneman_0011.jpg


In [ ]:
! git add .
! git config --global user.email "charliemeyers@gmail.com"
! git config --global user.name "simplymathematics"
! git commit -m "app"
! git push origin master

In [214]:
!ls

download.sh  facism.ipynb			  lfw-subset
facism	     haarcascade_frontalface_default.xml
